In [179]:
# Fix: Add list handling to expand_spec function
from itertools import product, combinations, permutations
from collections.abc import Mapping
import random

def expand_spec_fixed(node):
    # NEW: Handle lists by expanding each element and taking the product
    if isinstance(node, list):
        if not node:
            return [[]]  # Empty list -> single empty result

        # Special case: if there's only one element in the list, check if we should unwrap
        if len(node) == 1:
            element_result = expand_spec_fixed(node[0])

            # If the result contains lists (combinations), return directly to prevent extra wrapping
            # If the result contains scalars, we still need to wrap them properly
            if element_result and isinstance(element_result[0], list):
                return element_result
            # Otherwise, fall through to normal processing

        # Expand each element in the list
        expanded_elements = [expand_spec_fixed(element) for element in node]

        # Take Cartesian product of all expansions
        results = []
        for combo in product(*expanded_elements):
            results.append(list(combo))  # Convert tuple to list
        return results

    # Rest of the logic remains the same for dictionaries
    if not isinstance(node, Mapping):
        return [node]

    # Case 1: pure OR node (with optional size, count)
    if set(node.keys()).issubset({"or", "size", "count"}):
        choices = node["or"]
        size = node.get("size", None)
        count = node.get("count", None)

        if size is not None:
            # NEW: Check for second-order array notation [outer, inner]
            if isinstance(size, list) and len(size) == 2:
                results = _handle_nested_combinations(choices, size)
                # Apply count limit if specified
                if count is not None and len(results) > count:
                    results = random.sample(results, count)
                return results

            # Apply size constraints first, then expand
            out = []

            # Handle tuple size (from, to) or single size
            if isinstance(size, tuple) and len(size) == 2:
                from_size, to_size = size
                # Generate combinations for all sizes from from_size to to_size (inclusive)
                for s in range(from_size, to_size + 1):
                    if s > len(choices):
                        continue
                    for combo in combinations(choices, s):
                        # Expand this specific combination and return as individual elements
                        combo_results = _expand_combination(combo)
                        out.extend(combo_results)
            else:
                # Single size value
                if size <= len(choices):
                    for combo in combinations(choices, size):
                        # Expand this specific combination and return as individual elements
                        combo_results = _expand_combination(combo)
                        out.extend(combo_results)

            # Apply count limit if specified
            if count is not None and len(out) > count:
                out = random.sample(out, count)

            return out
        else:
            # Original behavior: expand all choices
            out = []
            for choice in choices:
                out.extend(expand_spec_fixed(choice))

            # Apply count limit if specified
            if count is not None and len(out) > count:
                out = random.sample(out, count)

            return out

    # Case 2: dict that also contains "or" -> branch and merge
    if "or" in node:
        # Extract size and count if present
        size = node.get("size", None)
        count = node.get("count", None)
        base = {k: v for k, v in node.items() if k not in ["or", "size", "count"]}
        base_expanded = expand_spec_fixed(base)            # list[dict]

        # Create a temporary or node with size and count
        or_node = {"or": node["or"]}
        if size is not None:
            or_node["size"] = size
        if count is not None:
            or_node["count"] = count

        choice_expanded = expand_spec_fixed(or_node)  # list[dict or scalar]
        results = []
        for b in base_expanded:
            for c in choice_expanded:
                if isinstance(c, Mapping):
                    merged = {**b, **c}
                    results.append(merged)
                else:
                    # Scalar choices only make sense as values under a key, not top-level merges
                    raise ValueError("Top-level 'or' choices must be dicts.")
        return results

    # Case 3: normal dict -> product over keys
    keys, options = zip(*[(k, _expand_value_fixed(v)) for k, v in node.items()]) if node else ([], [])
    if not keys:
        return [{}]
    out = []
    for combo in product(*options):
        d = {}
        for k, v in zip(keys, combo):
            d[k] = v
        out.append(d)
    return out

def _handle_nested_combinations(choices, nested_size):
    """Handle second-order combinations with array syntax [outer, inner]
    outer and inner can be int or tuple (from, to)
    For second-order: inner elements use permutations (order matters within sub-arrays)
    For outer selection: uses combinations (order doesn't matter for selecting which sub-arrays)
    """
    outer_size, inner_size = nested_size

    # Handle inner size - can be int or tuple
    if isinstance(inner_size, int):
        inner_from, inner_to = inner_size, inner_size
    else:  # tuple (from, to)
        inner_from, inner_to = inner_size

    # Handle outer size - can be int or tuple
    if isinstance(outer_size, int):
        outer_from, outer_to = outer_size, outer_size
    else:  # tuple (from, to)
        outer_from, outer_to = outer_size

    # Step 1: Generate all possible inner arrangements (permutations - order matters!)
    inner_arrangements = []
    for inner_s in range(inner_from, inner_to + 1):
        if inner_s > len(choices):
            continue
        # Use permutations for inner elements (order matters within sub-arrays)
        for perm in permutations(choices, inner_s):
            # Each inner arrangement is a simple list
            if len(perm) == 1:
                # Single element - don't wrap in extra list
                inner_arrangements.append(perm[0])
            else:
                # Multiple elements - keep as list
                inner_arrangements.append(list(perm))

    # Step 2: Select outer combinations (combinations - order doesn't matter for selection)
    out = []
    for outer_s in range(outer_from, outer_to + 1):
        if outer_s > len(inner_arrangements):
            continue
        # Use combinations for outer selection (order doesn't matter for which sub-arrays to pick)
        for outer_combo in combinations(inner_arrangements, outer_s):
            # Convert tuple to list and add to results
            out.append(list(outer_combo))

    return out

def _expand_combination(combo):
    """Expand a specific combination of choices by taking their cartesian product."""
    expanded_choices = []
    for choice in combo:
        expanded_choices.append(expand_spec_fixed(choice))

    # Take cartesian product
    results = []
    for expanded_combo in product(*expanded_choices):
        # Return the combination as a single list (flatten one level)
        results.append(list(expanded_combo))

    return results

def _expand_value_fixed(v):
    # Value position returns a list of *values* (scalars or dicts)
    if isinstance(v, Mapping) and ("or" in v.keys()):
        # Value-level OR can yield scalars or dicts as values (with optional size, count)
        choices = v["or"]
        size = v.get("size", None)
        count = v.get("count", None)

        if size is not None:
            # NEW: Check for second-order array notation [outer, inner]
            if isinstance(size, list) and len(size) == 2:
                results = _handle_nested_combinations(choices, size)
                # Apply count limit if specified
                if count is not None and len(results) > count:
                    results = random.sample(results, count)
                return results

            # Apply size constraints first, then expand
            vals = []

            # Handle tuple size (from, to) or single size
            if isinstance(size, tuple) and len(size) == 2:
                from_size, to_size = size
                # Generate combinations for all sizes from from_size to to_size (inclusive)
                for s in range(from_size, to_size + 1):
                    if s > len(choices):
                        continue
                    for combo in combinations(choices, s):
                        # Expand this specific combination
                        combo_results = _expand_combination(combo)
                        vals.extend(combo_results)
            else:
                # Single size value
                if size <= len(choices):
                    for combo in combinations(choices, size):
                        # Expand this specific combination
                        combo_results = _expand_combination(combo)
                        vals.extend(combo_results)

            # Apply count limit if specified
            if count is not None and len(vals) > count:
                vals = random.sample(vals, count)

            return vals
        else:
            # Original behavior: expand all choices
            vals = []
            for choice in choices:
                ex = expand_spec_fixed(choice)
                # expand_spec returns list; extend with each item (scalar or dict value)
                vals.extend(ex)

            # Apply count limit if specified
            if count is not None and len(vals) > count:
                vals = random.sample(vals, count)

            return vals
    elif isinstance(v, Mapping):
        # Nested object: expand to list of dict values
        return expand_spec_fixed(v)
    elif isinstance(v, list):
        # Handle lists in value positions
        return expand_spec_fixed(v)
    else:
        return [v]

In [180]:
pipeline_config = [
        {"or": [None, "A", "B"]},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results = expand_spec_fixed(pipeline_config)
print(f"Number of combinations (24): {len(results)}")
# for i, cfg in enumerate(results):
    # print(f"  {i+1}: {cfg}")

Number of combinations (24): 24


In [181]:
pipeline_config_with_size = [
        {"or": [None, "A", "B", "C", "D"], "size": 4},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results_with_size = expand_spec_fixed(pipeline_config_with_size)
# print(f"Expected: C(5,3) = 10 c(5,4) = 5")
print(f"Number of combinations (40): {len(results_with_size)}")
# for i, cfg in enumerate(results_with_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations (40): 40


In [182]:
pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", "D"], "size": (4, 5)},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
# print(f"Expected: C(5,3) + C(5,4) + C(5,5) = 10 + 5 + 1 = 16 combinations (before multiplying by other expansions)")
print(f"Number of combinations (48): {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations (48): 48


In [183]:
pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", {"or": ["a", "b"]}], "size": 4},  # scale the features
    ]
results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
print(f"Number of combinations: {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", {"or": ["a", "b"]}], "size": (3, 4)},  # scale the features
    ]
results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
print(f"Number of combinations: {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations: 9
Number of combinations: 25


In [184]:
p = [{"or": ["A", "B", "C"], "size": (1, 3)}]
r = expand_spec_fixed(p)
for item in r:
    print(item)

p = [{"or": ["A", "B", "C"]}]
r = expand_spec_fixed(p)
for item in r:
    print(item)

['A']
['B']
['C']
['A', 'B']
['A', 'C']
['B', 'C']
['A', 'B', 'C']
['A']
['B']
['C']


In [185]:
# Second-order combinations: [outer, inner] where inner uses permutations
config = [{"or": ["A", "B", "C"], "size": [2, 2]}]  # outer=2, inner=2
results = expand_spec_fixed(config)

print(f"Second-order [2, 2] results: {len(results)}")
for i, result in enumerate(results):
    print(f"  {i+1}: {result}")
# Expected: Inner permutations like ['A','B'] and ['B','A'] are different

Second-order [2, 2] results: 15
  1: [['A', 'B'], ['A', 'C']]
  2: [['A', 'B'], ['B', 'A']]
  3: [['A', 'B'], ['B', 'C']]
  4: [['A', 'B'], ['C', 'A']]
  5: [['A', 'B'], ['C', 'B']]
  6: [['A', 'C'], ['B', 'A']]
  7: [['A', 'C'], ['B', 'C']]
  8: [['A', 'C'], ['C', 'A']]
  9: [['A', 'C'], ['C', 'B']]
  10: [['B', 'A'], ['B', 'C']]
  11: [['B', 'A'], ['C', 'A']]
  12: [['B', 'A'], ['C', 'B']]
  13: [['B', 'C'], ['C', 'A']]
  14: [['B', 'C'], ['C', 'B']]
  15: [['C', 'A'], ['C', 'B']]


In [186]:
# Compare first-order vs second-order
choices = ["A", "B"]

# First-order: combinations (order doesn't matter)
config1 = [{"or": choices, "size": 2}]
results1 = expand_spec_fixed(config1)
print("First-order size=2:")
for i, result in enumerate(results1):
    print(f"  {i+1}: {result}")

# Second-order: permutations within inner (order matters)
config2 = [{"or": choices, "size": [1, 2]}]  # outer=1, inner=2
results2 = expand_spec_fixed(config2)
print(f"\nSecond-order [1, 2]:")
for i, result in enumerate(results2):
    print(f"  {i+1}: {result}")
# Expected: Both ['A','B'] and ['B','A'] appear

First-order size=2:
  1: ['A', 'B']

Second-order [1, 2]:
  1: [['A', 'B']]
  2: [['B', 'A']]


In [191]:
# Test inner permutations: [A, [B,C]] != [A, [C,B]]
config = [{"or": ["A", "B", "C"], "size": [(1,2), 2]}]
results = expand_spec_fixed(config)

print("Testing inner permutations:")
for i, result in enumerate(results):
    print(f"  {i+1}: {result}")
# Expected: Different orderings within sub-arrays

Testing inner permutations:
  1: [['A', 'B']]
  2: [['A', 'C']]
  3: [['B', 'A']]
  4: [['B', 'C']]
  5: [['C', 'A']]
  6: [['C', 'B']]
  7: [['A', 'B'], ['A', 'C']]
  8: [['A', 'B'], ['B', 'A']]
  9: [['A', 'B'], ['B', 'C']]
  10: [['A', 'B'], ['C', 'A']]
  11: [['A', 'B'], ['C', 'B']]
  12: [['A', 'C'], ['B', 'A']]
  13: [['A', 'C'], ['B', 'C']]
  14: [['A', 'C'], ['C', 'A']]
  15: [['A', 'C'], ['C', 'B']]
  16: [['B', 'A'], ['B', 'C']]
  17: [['B', 'A'], ['C', 'A']]
  18: [['B', 'A'], ['C', 'B']]
  19: [['B', 'C'], ['C', 'A']]
  20: [['B', 'C'], ['C', 'B']]
  21: [['C', 'A'], ['C', 'B']]


In [188]:
# Summary of all features
examples = [
    ("Basic or", [{"or": ["A", "B", "C"]}]),
    ("Size", [{"or": ["A", "B", "C", "D"], "size": 2}]),
    ("Size range", [{"or": ["A", "B", "C", "D"], "size": (2, 3)}]),
    ("Second-order", [{"or": ["A", "B", "C"], "size": [2, 2]}]),
    ("With count", [{"or": ["A", "B", "C", "D"], "size": (2, 3), "count": 5}]),
]

for name, config in examples:
    result = expand_spec_fixed(config)
    print(f"{name:12}: {len(result):4} combinations")

Basic or    :    3 combinations
Size        :    6 combinations
Size range  :   10 combinations
Second-order:   15 combinations
With count  :    5 combinations


In [194]:
# Count feature: random sampling
config = [{"or": ["A", "B", "C", "D"], "size": (2, 3), "count": 5}]
results = expand_spec_fixed(config)

print("Random 5 combinations from size (2,3):")
for i, result in enumerate(results):
    print(f"  {i+1}: {result}")
# Expected: Random 5 from total 16 combinations

Random 5 combinations from size (2,3):
  1: ['A', 'C']
  2: ['A', 'C', 'D']
  3: ['B', 'C']
  4: ['A', 'B', 'C']
  5: ['A', 'B', 'D']


In [ ]:
# Second-order with count
config = [{"or": ["A", "B", "C", "D"], "size": [2, 2], "count": 4}]
results = expand_spec_fixed(config)

print("Second-order [2, 2] with count=4:")
for i, result in enumerate(results):
    print(f"  {i+1}: {result}")
# Expected: Random 4 from many permutation-based combinations

Second-order [2, 2] with count=4:
  1: ['A', ['C', 'D', 'B']]
  2: [['A', 'D'], ['D', 'B', 'A']]
  3: [['B', 'A', 'C'], ['C', 'D', 'A']]
  4: [['C', 'A'], ['B', 'C', 'D']]


In [202]:
# Second-order with count
config = [{"or": ["A", "B", "C", "D"], "size": [3, (1,4)]}]#, "count": 4}]
results = expand_spec_fixed(config)

print(len(results), "results")
# print 10 random results
for i, result in enumerate(random.sample(results, min(10, len(results)))):
    print(f"  {i+1}: {result}")

# for i, result in enumerate(results):
    # print(f"  {i+1}: {result}")
# Expected: Random 4 from many permutation-based combinations

41664 results
  1: [['C', 'A'], ['A', 'D', 'B', 'C'], ['D', 'B', 'C', 'A']]
  2: [['C', 'B'], ['B', 'D', 'A'], ['A', 'D', 'B', 'C']]
  3: ['C', ['D', 'A', 'C'], ['B', 'A', 'C', 'D']]
  4: ['A', 'C', ['A', 'D']]
  5: [['D', 'A', 'C'], ['D', 'C', 'B'], ['B', 'D', 'C', 'A']]
  6: ['B', ['B', 'A'], ['B', 'C', 'D']]
  7: ['C', ['A', 'C', 'D', 'B'], ['D', 'C', 'B', 'A']]
  8: ['B', ['C', 'B'], ['D', 'C', 'A']]
  9: [['B', 'D'], ['D', 'C', 'A'], ['A', 'D', 'C', 'B']]
  10: [['B', 'A'], ['C', 'A', 'D'], ['C', 'B', 'A']]


## ✅ FINAL SYNTAX REFERENCE

### Basic Features:
- `{"or": ["A", "B", "C"]}` → All choices
- `{"or": ["A", "B", "C"], "size": 2}` → Combinations of size 2  
- `{"or": ["A", "B", "C"], "size": (1, 3)}` → Combinations of size 1 to 3
- `{"or": ["A", "B", "C"], "count": 5}` → Random 5 choices

### Second-Order (Array Syntax):
- `{"or": ["A", "B", "C"], "size": [outer, inner]}`
- **Inner**: Uses **permutations** (order matters within sub-arrays)  
- **Outer**: Uses **combinations** (order doesn't matter for selection)
- **Key**: `['A', 'B']` ≠ `['B', 'A']` within inner arrays

### Examples:
- `[2, 2]`: Select 2 arrangements of 2 elements each
- `[(1,3), 2]`: Select 1-3 arrangements of exactly 2 elements  
- `[2, (1,3)]`: Select exactly 2 arrangements of 1-3 elements
- `[2, 2, "count": 4]`: Random 4 from all second-order combinations